# データ前処理

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import datetime
import os
import calendar

In [3]:
import utils

In [4]:
OUT_DIRECTORY = './processed'

In [5]:
os.makedirs(OUT_DIRECTORY, exist_ok=True)

In [6]:
symbols = ['usdjpy', 'eurusd']

In [20]:
start_year = 2012
start_month = 4
# start_year = 2022
# start_month = 2

In [21]:
now = datetime.datetime.now(tz=datetime.timezone.utc)
current_year = now.year
current_month = now.month

In [24]:
def validate_data(df, symbol):
    """
    データの妥当性を検証する
    """

    FLAT_RATIO_TOLERANCE = 0.1
    NO_MOVE_RATIO_TOLERANCE = 0.1
    BID_HIGHER_RATIO_TOLERANCE = 0.0

    # フラット期間が一定割合以下
    flat_idxs = np.nonzero(np.all(df.iloc[1:].values == df.iloc[:-1].values, axis=1))[0]
    flat_ratio = len(flat_idxs) / len(df)
    assert flat_ratio <= FLAT_RATIO_TOLERANCE, f"flat_ratio = {flat_ratio} > {FLAT_RATIO_TOLERANCE}"

    # 4値同一が一定割合以下
    no_move_mask = (df["bid_high"] == df["bid_low"]) | (df["ask_high"] == df["ask_low"])
    no_move_ratio = no_move_mask.mean()
    assert no_move_ratio <= NO_MOVE_RATIO_TOLERANCE, f"no_move_ratio = {no_move_ratio} > {NO_MOVE_RATIO_TOLERANCE}"

    # bid > ask が一定割合以下
    bid_higher_mask = (
        (df["bid_open"] > df["ask_open"]) |
        (df["bid_high"] > df["ask_high"]) |
        (df["bid_low"] > df["ask_low"]) |
        (df["bid_close"] > df["ask_close"])
    )
    bid_higher_ratio = bid_higher_mask.mean()
    assert bid_higher_ratio <= BID_HIGHER_RATIO_TOLERANCE, f"bid_higer_ratio = {bid_higher_ratio} > {BID_HIGHER_RATIO_TOLERANCE}"
    
    # low < open, close < high の順になっている
    invalid_order_mask = (
        (df["bid_open"] < df["bid_low"]) |
        (df["bid_close"] < df["bid_low"]) |
        (df["bid_open"] > df["bid_high"]) |
        (df["bid_close"] > df["bid_high"]) |
        (df["ask_open"] < df["ask_low"]) |
        (df["ask_close"] < df["ask_low"]) |
        (df["ask_open"] > df["ask_high"]) |
        (df["ask_close"] > df["ask_high"])
    )
    assert invalid_order_mask.sum() == 0
    
    if symbol == "usdjpy":
        extreme_value_mask = (df < 70) | (df > 150)
        assert (extreme_value_mask.sum() == 0).all()
    elif symbol == "eurusd":
        extreme_value_mask = (df < 0.8) | (df > 1.6)
        assert (extreme_value_mask.sum() == 0).all()

In [25]:
for symbol in symbols:
    # 元データファイルは UTC+0 基準で保存されているので, UTC+2/+3 に合わせるために前月のデータが2/3時間分だけ必要
    prev_year, prev_month = utils.calc_year_month_offset(start_year, start_month, month_offset=-1)
    df_source = utils.read_raw_data(symbol, prev_year, prev_month, convert_timezone=True)
    df_source = df_source.loc[f"{start_year}-{start_month}"]

    year = start_year
    month = start_month

    while year < current_year or month < current_month:
        next_year, next_month = utils.calc_year_month_offset(year, month, month_offset=1)

        # 新しいデータを追加
        df_source = pd.concat([
            df_source,
            utils.read_raw_data(symbol, year, month, convert_timezone=True),
        ])

        # 当月データを抽出
        year_month_str = f"{year}-{month:02d}"
        df = df_source.loc[year_month_str]
        df = utils.remove_flat_data(df)

        validate_data(df, symbol)

        # 保存
        df.to_pickle(f"{OUT_DIRECTORY}/{symbol}-{year_month_str}.pkl")
        print(f"{symbol}: {year_month_str}")

        # 古いデータを削除
        df_source = df_source.loc[f"{next_year}-{next_month}"]

        year = next_year
        month = next_month

usdjpy: 2012-04
usdjpy: 2012-05
usdjpy: 2012-06
usdjpy: 2012-07
usdjpy: 2012-08
usdjpy: 2012-09
usdjpy: 2012-10
usdjpy: 2012-11
usdjpy: 2012-12
usdjpy: 2013-01
usdjpy: 2013-02
usdjpy: 2013-03
usdjpy: 2013-04
usdjpy: 2013-05
usdjpy: 2013-06
usdjpy: 2013-07
usdjpy: 2013-08
usdjpy: 2013-09
usdjpy: 2013-10
usdjpy: 2013-11
usdjpy: 2013-12
usdjpy: 2014-01
usdjpy: 2014-02
usdjpy: 2014-03
usdjpy: 2014-04
usdjpy: 2014-05
usdjpy: 2014-06
usdjpy: 2014-07
usdjpy: 2014-08
usdjpy: 2014-09
usdjpy: 2014-10
usdjpy: 2014-11
usdjpy: 2014-12
usdjpy: 2015-01
usdjpy: 2015-02
usdjpy: 2015-03
usdjpy: 2015-04
usdjpy: 2015-05
usdjpy: 2015-06
usdjpy: 2015-07
usdjpy: 2015-08
usdjpy: 2015-09
usdjpy: 2015-10
usdjpy: 2015-11
usdjpy: 2015-12
usdjpy: 2016-01
usdjpy: 2016-02
usdjpy: 2016-03
usdjpy: 2016-04
usdjpy: 2016-05
usdjpy: 2016-06
usdjpy: 2016-07
usdjpy: 2016-08
usdjpy: 2016-09
usdjpy: 2016-10
usdjpy: 2016-11
usdjpy: 2016-12
usdjpy: 2017-01
usdjpy: 2017-02
usdjpy: 2017-03
usdjpy: 2017-04
usdjpy: 2017-05
usdjpy: 